In [1]:
import math
import pandas as pd
from datetime import datetime, timedelta, date, time

### Universal Epoch date in astronomy - J2000 (Julian years)

In [2]:
# Convert 01.01.2000 @ 00:00 UT to datetime
d = date(2000,1,1)
t = time(0, 0)
j2000 = datetime.combine(d,t)

In [3]:
j2000

datetime.datetime(2000, 1, 1, 0, 0)

### User input

In [4]:
# Enter day, month and year in numerical form
day = int(input("Please enter your day of birth: "))
month = int(input("Please enter your month of birth: "))
year = int(input("Please enter your year of birth: "))

Please enter your day of birth: 1
Please enter your month of birth: 4
Please enter your year of birth: 2020


In [5]:
# (12,30)
t = int(input("Please enter your time of birth in HHMM format: "))

Please enter your time of birth in HHMM format: 0000


In [6]:
usr_time = time(t)

In [7]:
usr_dob = date(year, month, day)

In [8]:
usr_data = datetime.combine(usr_dob, usr_time)

In [9]:
usr_data

datetime.datetime(2020, 4, 1, 0, 0)

In [10]:
# Does the calculation change for dates before j2000? 
if j2000 > usr_data:
    yrs_since_epoch = j2000 - usr_data
else:
    yrs_since_epoch = usr_data - j2000

In [11]:
yrs_since_epoch

datetime.timedelta(days=7396)

In [12]:
# convert datetime epoch to int
days_since_epoch = yrs_since_epoch.days
print(type(yrs_since_epoch))

<class 'datetime.timedelta'>


In [13]:
# Celestial coordinates at epoch 01.01.2000 @ 00:00 UT
df ={'Planet':['Mercury','Venus','Earth','Mars','Jupiter','Saturn'],'λi':[250.2,181.2,100.0,355.2,34.3,50.1],'T_(Days)':[87.969,224.701,365.256,686.980,4332.59,10759.2],'ω0':[4.09235,1.60213,0.98561,0.52403,0.08308,0.03346],'λp':[77.5,131.6,102.9,336.1,14.3,93.1],'e':[0.2056,0.0068,0.0167,0.0934,0.0485,0.0555], 'a (A.U.)':[0.387,0.723,1.0,1.52,5.2,9.55]}

In [14]:
df = pd.DataFrame(data=df)

In [15]:
df

,Planet,λi,T_(Days),ω0,λp,e,a (A.U.)
0,Mercury,250.2,87.969,4.09235,77.5,0.2056,0.387
1,Venus,181.2,224.701,1.60213,131.6,0.0068,0.723
2,Earth,100.0,365.256,0.98561,102.9,0.0167,1.000
3,Mars,355.2,686.980,0.52403,336.1,0.0934,1.520
4,Jupiter,34.3,4332.590,0.08308,14.3,0.0485,5.200
5,Saturn,50.1,10759.200,0.03346,93.1,0.0555,9.550


### Function to calculate heliocentric longitude 

In [45]:
planet = "Saturn"

In [48]:
def helio_longitude(df, planet, days_since_epoch):
    'Calculate heliocentric longitude for a given planet'
    # Circular orbit
    init_mean_long = df.loc[df.Planet == planet, "λi"].iloc[0]
    angle_trav_per_day = df.loc[df.Planet == planet, "ω0"].iloc[0]
    mean_ang_traversed = angle_trav_per_day * days_since_epoch
    mean_long = init_mean_long + mean_ang_traversed
    helio_mean_long = round(mean_long % 360,2)
    # Convert circular to heliocentric orbit
    perihelion_long = df.loc[df.Planet == planet, "λp"].iloc[0]
    eccentricity = df.loc[df.Planet == planet, "e"].iloc[0]
    mean_anomaly = helio_mean_long - perihelion_long
    helio_ellip = math.degrees(2*eccentricity*(math.sin(math.radians(mean_anomaly)))) + math.degrees((1.25 * eccentricity**2*math.sin(math.radians(2*mean_anomaly))))
    frac_of_year = round(days_since_epoch / 365.25,2)
    prec_vern_eqnx = round(frac_of_year * 360/25800,2)
    geo_longitude = helio_mean_long + helio_ellip + prec_vern_eqnx
    return geo_longitude

In [47]:
helio_longitude(df, planet, days_since_epoch)

295.3819936777624

### Calculate Mars heliocentric circular orbit for inputted user DOB

In [31]:
init_mean_long = df.loc[df.Planet == "Mars", "λi"].iloc[0] 
init_mean_long

355.2

In [79]:
angle_trav_per_day = df.loc[df.Planet == "Mars", "ω0"].iloc[0]
angle_trav_per_day

0.52403

In [93]:
mean_ang_traversed = angle_trav_per_day * days_since_epoch
mean_ang_traversed

3875.72588

In [94]:
mean_long = init_mean_long + mean_ang_traversed
mean_long

4230.92588

In [134]:
helio_mean_long = round(mean_long % 360,2)
helio_mean_long

270.93

### Correct Mars circular to Heliocentric elliptical orbit

In [135]:
# Formula to correct for elliptical orbit
perihelion_long = df.loc[df.Planet == "Mars", "λp"].iloc[0]
eccentricity = df.loc[df.Planet == "Mars", "e"].iloc[0]

In [136]:
helio_mean_long

270.93

In [137]:
mean_anomaly = helio_mean_long - perihelion_long

print(perihelion_long)
print(mean_anomaly)
print(eccentricity)

336.1
-65.17000000000002
0.0934


In [138]:
helio_ellip = math.degrees(2*eccentricity*(math.sin(math.radians(mean_anomaly)))) + math.degrees((1.25 * eccentricity**2*math.sin(math.radians(2*mean_anomaly))))
helio_ellip

-10.189672605719439

In [139]:
# Precession of vernal equinox from time of user's DOB in years
frac_of_year = round(days_since_epoch / 365.25,2)
prec_vern_eqnx = round(frac_of_year * 360/25800,2)
prec_vern_eqnx

0.28

In [140]:
helio_mean_long + helio_ellip + precession_v_eq

261.0203273942805

### Correct Mercury circular to Heliocentric elliptical orbit

In [164]:
# Formula to correct for elliptical orbit
perihelion_long = df.loc[df.Planet == "Mercury", "λp"].iloc[0]
eccentricity = df.loc[df.Planet == "Mercury", "e"].iloc[0]

# mean_anomaly = helio_mean_long - perihelion_long 
helio_mean_long = 277.2 #Mercury
mean_anomaly = helio_mean_long - perihelion_long

print(perihelion_long)
print(mean_anomaly)
print(eccentricity)

77.5
199.7
0.2056


In [165]:
helio_ellip = math.degrees(2*eccentricity*(math.sin(math.radians(mean_anomaly)))) + math.degrees((1.25 * eccentricity**2*math.sin(math.radians(2*mean_anomaly))))
helio_ellip

-6.020349318634777

In [166]:
# Precession of vernal equinox from time of user's DOB in years
frac_of_year = round(days_since_epoch / 365.25,2)
prec_vern_eqnx = round(frac_of_year * 360/25800,2)
prec_vern_eqnx

0.28

In [167]:
# Should be 261.0 for Mars
# Longitude of vernal equinox for Mars = mean longitude + mean anomaly + precession of vernal equinox
helio_mean_long + helio_ellip + precession_v_eq

271.4596506813652

## Function to generate helio_mean_long from all planets

### Calculate Geocentric longitude

1. Calculate radii, r, of planet's orbit around the sun, for that epoch, giving their positions in polar form ((r, λ)

### Getting Mercury's geocentric coordinates

In [155]:
a = df.loc[df.Planet == "Mercury", "a (A.U.)"].iloc[0]
a

0.387

In [157]:
e = df.loc[df.Planet == "Mercury", "e"].iloc[0]
e

0.2056

In [160]:
r = a*(1-e**2)/1+(e*math.cos(mean_anomaly))

In [161]:
r

0.4132723832527231

In [168]:
helio_polar_coord = [r, helio_mean_long]

In [169]:
helio_polar_coord

[0.4132723832527231, 277.2]

### Find r_AU of Sun/Earth

In [ ]:
e = df.loc[df.Planet == "Earth", "e"].iloc[0]
r_earth = a*(1-e**2)/1+(e*math.cos(mean_anomaly))